# Introduction
The purpose of this notebook is to provide an example of data cleaning and preparation. In this example, data will be pulled from the Social Security's data set of popular baby names found [here](https://www.ssa.gov/oact/babynames/limits.html). This data will be directly downloaded from the website and organized into one data frame that will be used for a streamlit app.

# Downloading the Data
First, we'll pull the data directly from the website found [here](https://www.ssa.gov/oact/babynames/limits.html). At first, I thought this was going to be a simple download using the code below...

In [2]:
import requests

# Basic download
url = "https://www.ssa.gov/oact/babynames/names.zip"
response = requests.get(url)

# Save to file
with open("downloaded_file.zip", "wb") as f:
    f.write(response.content)

# Check for errors
if response.status_code == 200:
    print("Download successful")
else:
    print(f"Error: {response.status_code}")

Error: 403


I was wrong. After, many code re-writes it looks like this website detects automated requests and blocks them. I'll try to get around this by using Selenium to simulate being an actual user. NOTE: This should be done with caution. This is a sure way to get kicked off a website if you abuse this power. This happened to me during one of my projects at Booz Allen. I used Selenium to webscrape [matweb](https://matweb.com/) and aggregate material information that I needed for the project. I learned the hard way that your IP address will be black listed if you extract too much data. For this project though, I just want to download one link and that's it. So let's begin. 

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os

def download_with_selenium_custom_folder(redownload=False):
    if redownload==False:
        if os.path.exists(os.path.abspath("data/names.zip")):
            print("File already exists. Skipping download.")
            return
            
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
    
    # Set download directory
    download_dir = os.path.abspath("data")  # Downloads to your project's data folder
    os.makedirs(download_dir, exist_ok=True)
    
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    
    try:
        # Visit the main page first
        driver.get("https://www.ssa.gov/oact/babynames/limits.html")
        time.sleep(2)
        
        # Find and click the download link
        links = driver.find_elements(By.TAG_NAME, "a")
        for link in links:
            href = link.get_attribute("href")
            if href and "names.zip" in href:
                print(f"Downloading from: {href}")
                driver.get(href)
                time.sleep(5)  # Wait for download to complete
                print(f"File downloaded to: {download_dir}")
                break
                
    finally:
        driver.quit()

# Run the function
download_with_selenium_custom_folder()

File already exists. Skipping download.


Worked like a charm! Now it's time to restructre the data.

# Restructuring the Data
The data is in a zip folder so let's unzip it.

In [4]:
# unzip the file
import zipfile
zip_file_path = os.path.abspath("data/names.zip")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("data")

Looks like the data was all stored in text files and there is a ReadMe that states:

> **National Data on the relative frequency of given names in the population of
> U.S. births where the individual has a Social Security Number**
> (Tabulated based on Social Security records as of March 2, 2025)
> 
> For each year of birth YYYY after 1879, we created a comma-delimited file called yobYYYY.txt. Each
> record in the individual annual files has the format "name,sex,number," where name is 2 to 15 characters,
> sex is M (male) or F (female) and "number" is the number of occurrences of the name. Each file is sorted
> first on sex and then on number of occurrences in descending order. When there is a tie on the number of
> occurrences, names are listed in alphabetical order. This sorting makes it easy to determine a name's rank.
> The first record for each sex has rank 1, the second record for each sex has rank 2, and so forth.
>
> To safeguard privacy, we restrict our list of names to those with at least 5 occurrences

Alright, we'll have to read in all of the txt files and aggregate them into one pandas data frame.

In [5]:
# read in all of the txt files
import pandas as pd

# get all of the txt files in the data folder
txt_files = [f for f in os.listdir("data") if f.endswith(".txt")]

# read in all of the txt files
df = pd.DataFrame()
for file in txt_files:
    df_add = pd.read_csv(os.path.join("data", file), header=None)
    df_add.columns = ["name", "sex", "total_count"]
    df_add["year"] = file.split("yob")[1].split(".txt")[0]
    df = pd.concat([df, df_add])

# convert year to int
df["year"] = df["year"].astype(int)

# drop duplicates and reset index
df = df.drop_duplicates(subset=["name", "sex", "year"])
df = df.reset_index(drop=True)

# sort by year, sex, and total_count
df = df.sort_values(by=["year", "sex", "total_count"], ascending=False)

df.head()

,name,sex,total_count,year
2135234,Liam,M,22164,2024
2135235,Noah,M,20337,2024
2135236,Oliver,M,15343,2024
2135237,Theodore,M,12011,2024
2135238,James,M,11793,2024


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2149477 entries, 2135234 to 941
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   name         object
 1   sex          object
 2   total_count  int64 
 3   year         int64 
dtypes: int64(2), object(2)
memory usage: 82.0+ MB


Let's also update the data so names are persistent each year even if it's total count is zero for a year. For example, if a unique name like Zyler was use 2 times in 2023 but not once in 2024. There isn't a row stating the total count for Zyler was 0 for 2024. I want to include these rows to help perform trend analysis down the road. The reasoning for this might not be apparent now, but you will see why this is important when we start developing projections for popularities of names.

In [7]:
names_sex = df[['name', 'sex']].drop_duplicates()
years = pd.DataFrame({'year': df['year'].drop_duplicates()})

# Cross join (cartesian product)
all_names = names_sex.assign(key=1).merge(years.assign(key=1), on='key').drop('key', axis=1)
all_names['total_count'] = 0

# merge the data frames
df = pd.concat([df, all_names])

# drop duplicates and reset index
df = df.drop_duplicates(subset=["name", "sex", "year"])
df = df.reset_index(drop=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16899750 entries, 0 to 16899749
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   name         object
 1   sex          object
 2   total_count  int64 
 3   year         int64 
dtypes: int64(2), object(2)
memory usage: 515.7+ MB


Let's add a new feature that calculates the relative popularity of a name for each year.

In [8]:
# calculate the relative popularity of a name for each year
df["popularity_percent"] = df.groupby(["sex", "year"])["total_count"].transform("sum")
df["popularity_percent"] = df["total_count"] / df["popularity_percent"]

df['popularity_rank'] = df.groupby(['sex', 'year'])['popularity_percent'].rank(method='min', ascending=False).astype(int)
df.head()


,name,sex,total_count,year,popularity_percent,popularity_rank
0,Liam,M,22164,2024,0.012921,1
1,Noah,M,20337,2024,0.011856,2
2,Oliver,M,15343,2024,0.008945,3
3,Theodore,M,12011,2024,0.007002,4
4,James,M,11793,2024,0.006875,5


# Create SQLite Database
And now we have one giant data frame. Let's save it to a SQLite database and use it for the Streamlit app.

In [ ]:
import sqlite3

db_path = "data/names.db"
if os.path.exists(db_path)==False:
    # create a connection to the database
    conn = sqlite3.connect("data/names.db")

    # save the data frame to a SQLite database  
    df.to_sql("names", conn, if_exists="replace", index=False)

    # Create indexes for better performance
    cursor = conn.cursor()
    cursor.execute('CREATE INDEX idx_name ON names(name)')
    cursor.execute('CREATE INDEX idx_sex ON names(sex)')
    cursor.execute('CREATE INDEX idx_year ON names(year)')
    conn.commit()

    # close the connection
    conn.close()

# Machine Learning Models for Name Popularity Prediction

In this section, we'll create multiple machine learning models to predict the future popularity of baby names. We'll train several different models and compare their performance to determine which one works best for this time series prediction task.


In [ ]:
# Import necessary libraries for machine learning
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


## Data Preparation for Machine Learning

First, we need to prepare our data for time series prediction. We'll create features that capture temporal patterns and trends in name popularity.


In [ ]:
# Create a function to prepare data for a specific name and sex
def prepare_name_data(df, name, sex, lookback_years=5):
    """
    Prepare time series data for a specific name and sex combination.
    
    Parameters:
    - df: DataFrame with name data
    - name: Name to analyze
    - sex: Gender ('M' or 'F')
    - lookback_years: Number of years to look back for features
    
    Returns:
    - X: Feature matrix
    - y: Target variable (popularity_percent)
    - years: Corresponding years
    """
    
    # Filter data for specific name and sex
    name_data = df[(df['name'] == name) & (df['sex'] == sex)].copy()
    name_data = name_data.sort_values('year').reset_index(drop=True)
    
    if len(name_data) < lookback_years + 1:
        return None, None, None
    
    # Create features
    features = []
    targets = []
    years = []
    
    for i in range(lookback_years, len(name_data)):
        # Target: popularity_percent for current year
        target = name_data.iloc[i]['popularity_percent']
        
        # Features: historical data
        feature_row = []
        
        # Previous years' popularity_percent
        for j in range(lookback_years):
            feature_row.append(name_data.iloc[i - lookback_years + j]['popularity_percent'])
        
        # Previous years' total_count
        for j in range(lookback_years):
            feature_row.append(name_data.iloc[i - lookback_years + j]['total_count'])
        
        # Previous years' popularity_rank
        for j in range(lookback_years):
            feature_row.append(name_data.iloc[i - lookback_years + j]['popularity_rank'])
        
        # Trend features
        recent_popularity = name_data.iloc[i-lookback_years:i]['popularity_percent'].values
        recent_counts = name_data.iloc[i-lookback_years:i]['total_count'].values
        
        # Linear trend in popularity
        if len(recent_popularity) > 1:
            trend_pop = np.polyfit(range(len(recent_popularity)), recent_popularity, 1)[0]
        else:
            trend_pop = 0
        
        # Linear trend in counts
        if len(recent_counts) > 1:
            trend_count = np.polyfit(range(len(recent_counts)), recent_counts, 1)[0]
        else:
            trend_count = 0
        
        # Volatility (standard deviation of recent popularity)
        volatility = np.std(recent_popularity)
        
        # Add trend and volatility features
        feature_row.extend([trend_pop, trend_count, volatility])
        
        # Year as a feature (normalized)
        year_normalized = (name_data.iloc[i]['year'] - 1880) / (2024 - 1880)
        feature_row.append(year_normalized)
        
        features.append(feature_row)
        targets.append(target)
        years.append(name_data.iloc[i]['year'])
    
    return np.array(features), np.array(targets), years

# Test the function with a popular name
print("Testing data preparation with 'Liam' (Male):")
liam_data = prepare_name_data(df, 'Liam', 'M')
if liam_data[0] is not None:
    print(f"Feature matrix shape: {liam_data[0].shape}")
    print(f"Target array shape: {liam_data[1].shape}")
    print(f"Number of samples: {len(liam_data[2])}")
else:
    print("Insufficient data for 'Liam'")


In [ ]:
# Create a comprehensive dataset for training
def create_training_dataset(df, min_samples=10, lookback_years=5):
    """
    Create a comprehensive training dataset from all names with sufficient data.
    
    Parameters:
    - df: DataFrame with name data
    - min_samples: Minimum number of samples required for a name to be included
    - lookback_years: Number of years to look back for features
    
    Returns:
    - X: Combined feature matrix
    - y: Combined target array
    - name_sex_pairs: List of (name, sex) pairs for each sample
    """
    
    all_features = []
    all_targets = []
    name_sex_pairs = []
    
    # Get unique name-sex combinations
    unique_combinations = df[['name', 'sex']].drop_duplicates()
    
    print(f"Processing {len(unique_combinations)} name-sex combinations...")
    
    for idx, row in unique_combinations.iterrows():
        name = row['name']
        sex = row['sex']
        
        # Prepare data for this name-sex combination
        X, y, years = prepare_name_data(df, name, sex, lookback_years)
        
        if X is not None and len(X) >= min_samples:
            all_features.append(X)
            all_targets.append(y)
            # Add name-sex pair for each sample
            name_sex_pairs.extend([(name, sex)] * len(X))
            
        if idx % 1000 == 0:
            print(f"Processed {idx}/{len(unique_combinations)} combinations...")
    
    # Combine all data
    X_combined = np.vstack(all_features)
    y_combined = np.concatenate(all_targets)
    
    print(f"Final dataset shape: {X_combined.shape}")
    print(f"Target array shape: {y_combined.shape}")
    
    return X_combined, y_combined, name_sex_pairs

# Create the training dataset
X, y, name_sex_pairs = create_training_dataset(df, min_samples=5, lookback_years=5)


## Model Training and Evaluation

Now we'll train multiple machine learning models and compare their performance using appropriate time series cross-validation.


In [ ]:
# Split data for training and testing
# Use time-based split to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
}

# Train and evaluate models
model_results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train model
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    model_results[name] = {
        'model': model,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'predictions': y_pred
    }
    
    print(f"{name} Results:")
    print(f"  RMSE: {rmse:.6f}")
    print(f"  MAE: {mae:.6f}")
    print(f"  R²: {r2:.6f}")


In [ ]:
# Create LSTM model for time series prediction
def create_lstm_model(input_shape):
    """Create an LSTM model for time series prediction."""
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Prepare data for LSTM (needs 3D input: samples, timesteps, features)
def prepare_lstm_data(X, y, lookback_years=5):
    """Prepare data for LSTM model."""
    # Reshape X to (samples, timesteps, features)
    # We'll use the lookback_years as timesteps
    n_samples = X.shape[0]
    n_features = X.shape[1] // lookback_years  # Each timestep has this many features
    
    X_lstm = X.reshape(n_samples, lookback_years, n_features)
    return X_lstm, y

# Prepare LSTM data
X_lstm, y_lstm = prepare_lstm_data(X, y, lookback_years=5)
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(
    X_lstm, y_lstm, test_size=0.2, random_state=42
)

# Scale the LSTM data
scaler_lstm = StandardScaler()
X_train_lstm_scaled = scaler_lstm.fit_transform(X_train_lstm.reshape(-1, X_train_lstm.shape[-1]))
X_train_lstm_scaled = X_train_lstm_scaled.reshape(X_train_lstm.shape)

X_test_lstm_scaled = scaler_lstm.transform(X_test_lstm.reshape(-1, X_test_lstm.shape[-1]))
X_test_lstm_scaled = X_test_lstm_scaled.reshape(X_test_lstm.shape)

print(f"LSTM Training data shape: {X_train_lstm_scaled.shape}")
print(f"LSTM Test data shape: {X_test_lstm_scaled.shape}")

# Create and train LSTM model
lstm_model = create_lstm_model((X_train_lstm_scaled.shape[1], X_train_lstm_scaled.shape[2]))

print("\nTraining LSTM model...")
history = lstm_model.fit(
    X_train_lstm_scaled, y_train_lstm,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

# Make predictions
y_pred_lstm = lstm_model.predict(X_test_lstm_scaled, verbose=0).flatten()

# Calculate LSTM metrics
mse_lstm = mean_squared_error(y_test_lstm, y_pred_lstm)
rmse_lstm = np.sqrt(mse_lstm)
mae_lstm = mean_absolute_error(y_test_lstm, y_pred_lstm)
r2_lstm = r2_score(y_test_lstm, y_pred_lstm)

# Store LSTM results
model_results['LSTM'] = {
    'model': lstm_model,
    'mse': mse_lstm,
    'rmse': rmse_lstm,
    'mae': mae_lstm,
    'r2': r2_lstm,
    'predictions': y_pred_lstm
}

print(f"LSTM Results:")
print(f"  RMSE: {rmse_lstm:.6f}")
print(f"  MAE: {mae_lstm:.6f}")
print(f"  R²: {r2_lstm:.6f}")


## Model Performance Comparison

Let's compare the performance of all models and create visualizations to understand their strengths and weaknesses.


In [ ]:
# Create performance comparison
import matplotlib.pyplot as plt

# Create comparison DataFrame
comparison_data = []
for model_name, results in model_results.items():
    comparison_data.append({
        'Model': model_name,
        'RMSE': results['rmse'],
        'MAE': results['mae'],
        'R²': results['r2']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('RMSE')

print("Model Performance Comparison:")
print("=" * 50)
print(comparison_df.to_string(index=False, float_format='%.6f'))

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# RMSE comparison
axes[0, 0].bar(comparison_df['Model'], comparison_df['RMSE'], color='skyblue')
axes[0, 0].set_title('Root Mean Square Error (RMSE)')
axes[0, 0].set_ylabel('RMSE')
axes[0, 0].tick_params(axis='x', rotation=45)

# MAE comparison
axes[0, 1].bar(comparison_df['Model'], comparison_df['MAE'], color='lightcoral')
axes[0, 1].set_title('Mean Absolute Error (MAE)')
axes[0, 1].set_ylabel('MAE')
axes[0, 1].tick_params(axis='x', rotation=45)

# R² comparison
axes[1, 0].bar(comparison_df['Model'], comparison_df['R²'], color='lightgreen')
axes[1, 0].set_title('R² Score')
axes[1, 0].set_ylabel('R²')
axes[1, 0].tick_params(axis='x', rotation=45)

# Predictions vs Actual (using best model)
best_model_name = comparison_df.iloc[0]['Model']
best_predictions = model_results[best_model_name]['predictions']

axes[1, 1].scatter(y_test, best_predictions, alpha=0.5, s=1)
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 1].set_xlabel('Actual Values')
axes[1, 1].set_ylabel('Predicted Values')
axes[1, 1].set_title(f'Predictions vs Actual ({best_model_name})')

plt.tight_layout()
plt.show()


In [ ]:
# Additional analysis: Feature importance for tree-based models
def plot_feature_importance(model, model_name, feature_names=None):
    """Plot feature importance for tree-based models."""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        
        if feature_names is None:
            feature_names = [f'Feature_{i}' for i in range(len(importances))]
        
        # Get top 20 most important features
        indices = np.argsort(importances)[::-1][:20]
        
        plt.figure(figsize=(10, 8))
        plt.title(f'Feature Importance - {model_name}')
        plt.bar(range(len(indices)), importances[indices])
        plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

# Create feature names for interpretation
feature_names = []
lookback_years = 5

# Historical popularity_percent features
for i in range(lookback_years):
    feature_names.append(f'popularity_percent_t-{lookback_years-i-1}')

# Historical total_count features
for i in range(lookback_years):
    feature_names.append(f'total_count_t-{lookback_years-i-1}')

# Historical popularity_rank features
for i in range(lookback_years):
    feature_names.append(f'popularity_rank_t-{lookback_years-i-1}')

# Trend and volatility features
feature_names.extend(['trend_popularity', 'trend_count', 'volatility', 'year_normalized'])

print(f"Total features: {len(feature_names)}")
print("Feature names:", feature_names[:10], "...")

# Plot feature importance for Random Forest and XGBoost
for model_name in ['Random Forest', 'XGBoost']:
    if model_name in model_results:
        plot_feature_importance(model_results[model_name]['model'], model_name, feature_names)


## Model Assessment and Recommendations

Based on the performance metrics and analysis, let's provide a comprehensive assessment of which model should be used for predicting name popularity.


In [ ]:
# Comprehensive model assessment
def assess_model_performance(model_results):
    """Provide comprehensive assessment of model performance."""
    
    print("=" * 80)
    print("COMPREHENSIVE MODEL ASSESSMENT")
    print("=" * 80)
    
    # Sort models by RMSE (lower is better)
    sorted_models = sorted(model_results.items(), key=lambda x: x[1]['rmse'])
    
    print("\n1. PERFORMANCE RANKING (by RMSE):")
    print("-" * 40)
    for i, (name, results) in enumerate(sorted_models, 1):
        print(f"{i}. {name}:")
        print(f"   RMSE: {results['rmse']:.6f}")
        print(f"   MAE:  {results['mae']:.6f}")
        print(f"   R²:   {results['r2']:.6f}")
        print()
    
    # Best model
    best_model_name, best_results = sorted_models[0]
    
    print("2. BEST PERFORMING MODEL:")
    print("-" * 30)
    print(f"Model: {best_model_name}")
    print(f"RMSE: {best_results['rmse']:.6f}")
    print(f"MAE: {best_results['mae']:.6f}")
    print(f"R²: {best_results['r2']:.6f}")
    print()
    
    # Model characteristics analysis
    print("3. MODEL CHARACTERISTICS:")
    print("-" * 30)
    
    for name, results in model_results.items():
        print(f"\n{name}:")
        
        # Interpretability
        if name == 'Linear Regression':
            print("  • High interpretability - coefficients show feature importance")
            print("  • Assumes linear relationships")
        elif name in ['Random Forest', 'XGBoost']:
            print("  • Medium interpretability - feature importance available")
            print("  • Can capture non-linear relationships")
        elif name == 'LSTM':
            print("  • Low interpretability - black box model")
            print("  • Designed for sequential/time series data")
        
        # Training time and complexity
        if name == 'Linear Regression':
            print("  • Fast training and prediction")
            print("  • Low computational requirements")
        elif name in ['Random Forest', 'XGBoost']:
            print("  • Moderate training time")
            print("  • Good for handling mixed data types")
        elif name == 'LSTM':
            print("  • Slowest training time")
            print("  • Requires more data and computational resources")
    
    # Recommendations
    print("\n4. RECOMMENDATIONS:")
    print("-" * 20)
    
    print(f"\nPRIMARY RECOMMENDATION: {best_model_name}")
    print(f"Reason: Lowest RMSE ({best_results['rmse']:.6f}) indicates best overall accuracy")
    
    # Additional recommendations based on use case
    print("\nUSE CASE RECOMMENDATIONS:")
    print("• For quick predictions and interpretability: Linear Regression")
    print("• For robust predictions with feature insights: Random Forest or XGBoost")
    print("• For complex time series patterns: LSTM (if computational resources allow)")
    print("• For production deployment: XGBoost (good balance of performance and speed)")
    
    # Model limitations
    print("\n5. MODEL LIMITATIONS:")
    print("-" * 20)
    print("• All models assume historical patterns will continue")
    print("• External factors (cultural trends, events) not captured")
    print("• Predictions become less reliable further into the future")
    print("• Models may not handle sudden popularity spikes well")
    
    return best_model_name, best_results

# Run the assessment
best_model, best_results = assess_model_performance(model_results)


In [ ]:
# Save the best model and scaler for future use
import joblib

# Save the best model
best_model_obj = model_results[best_model]['model']
joblib.dump(best_model_obj, f'data/best_model_{best_model.lower().replace(" ", "_")}.pkl')

# Save the scaler
if best_model == 'Linear Regression':
    joblib.dump(scaler, 'data/scaler.pkl')
elif best_model == 'LSTM':
    joblib.dump(scaler_lstm, 'data/scaler_lstm.pkl')

# Save feature names for future reference
joblib.dump(feature_names, 'data/feature_names.pkl')

print(f"\n6. MODEL PERSISTENCE:")
print("-" * 20)
print(f"Best model ({best_model}) saved to: data/best_model_{best_model.lower().replace(' ', '_')}.pkl")
print("Scaler saved to: data/scaler.pkl")
print("Feature names saved to: data/feature_names.pkl")
print("\nThese files can be loaded in your Streamlit app for making predictions!")

# Create a prediction function for future use
def predict_name_popularity(name, sex, years_ahead=1, model_name=best_model):
    """
    Predict future popularity for a given name and sex.
    
    Parameters:
    - name: Name to predict
    - sex: Gender ('M' or 'F')
    - years_ahead: Number of years to predict ahead
    - model_name: Model to use for prediction
    
    Returns:
    - predictions: List of predicted popularity_percent values
    """
    
    # Get historical data for the name
    name_data = df[(df['name'] == name) & (df['sex'] == sex)].copy()
    name_data = name_data.sort_values('year').reset_index(drop=True)
    
    if len(name_data) < 5:
        return None, "Insufficient historical data"
    
    # Prepare features for prediction
    X_pred, _, _ = prepare_name_data(df, name, sex, lookback_years=5)
    
    if X_pred is None or len(X_pred) == 0:
        return None, "Could not prepare features"
    
    # Use the most recent data point for prediction
    latest_features = X_pred[-1:].copy()
    
    # Load the appropriate model and scaler
    model = joblib.load(f'data/best_model_{model_name.lower().replace(" ", "_")}.pkl')
    
    if model_name == 'Linear Regression':
        scaler = joblib.load('data/scaler.pkl')
        latest_features = scaler.transform(latest_features)
    elif model_name == 'LSTM':
        scaler = joblib.load('data/scaler_lstm.pkl')
        # Reshape for LSTM
        latest_features = latest_features.reshape(1, 5, -1)
        latest_features = scaler.transform(latest_features.reshape(-1, latest_features.shape[-1]))
        latest_features = latest_features.reshape(1, 5, -1)
    
    # Make prediction
    prediction = model.predict(latest_features)
    
    return prediction[0], "Success"

# Test the prediction function
print(f"\n7. PREDICTION FUNCTION TEST:")
print("-" * 30)
test_name = "Liam"
test_sex = "M"
prediction, status = predict_name_popularity(test_name, test_sex, years_ahead=1)

if status == "Success":
    print(f"Predicted popularity for {test_name} ({test_sex}) next year: {prediction:.6f}")
    print(f"Current popularity: {df[(df['name'] == test_name) & (df['sex'] == test_sex) & (df['year'] == 2024)]['popularity_percent'].iloc[0]:.6f}")
else:
    print(f"Prediction failed: {status}")
